<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>NEXT TONNE - BACKGROUND INDEX</p><br>

In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# General importings
import math
import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

# IC stuff
import invisible_cities.core.system_of_units as units

# TONNE stuff
from detector_dimensions  import get_dimensions
from detector_dimensions  import print_dimensions

from material_activities  import get_activities
from material_activities  import print_activities

from detector_backgrounds import get_background_level
from rejection_factors    import get_rejection_factors
from roi_settings         import get_roi_settings


# DETAILS OF ANALYSIS

In [5]:
# VERBOSITY
VERBOSE = True

# Available detectors: "next_2x2", "next_3x3", "next_hd"
DETECTOR = 'next_hd'

# Available background levels: "pessimistic", "probable", "optimistic"
BKGND_LEVEL = 'pessimistic'

# LOADING DETECTOR DATA

In [9]:
det_dim = get_dimensions(DETECTOR)

In [10]:
if VERBOSE:
    print_dimensions(DETECTOR)

*** DETECTOR 'next_hd' DIMENSIONS ***

* ACTIVE
  ACTIVE Diameter = 260.0 cm
  ACTIVE Length   = 260.0 cm
  ACTIVE Volume   = 1.38e+07 cm**3
  ACTIVE mass     = 1.229e+03 kg

* READOUT_PLANE
  READOUT_PLANE Surface = 5.309e+04 cm**2

* CATHODE
  CATHODE Volume = 1.327e+03 cm**3
  CATHODE Mass   = 10.61 kg

* FIELD_CAGE
  FIELD_CAGE Thickness = 1.0 cm
  FIELD_CAGE Inner Rad = 130.0 cm
  FIELD_CAGE Outer Rad = 131.0 cm
  FIELD_CAGE Length    = 260.0 cm
  FIELD_CAGE Volume    = 2.132e+05 cm**3
  FIELD_CAGE Mass      = 206.8 kg

* INNER_SHIELDING
  ICS Thickness         = 12.0 cm
  ICS Inner Rad         = 131.0 cm
  ICS Outer Rad         = 143.0 cm
  ICS Inner Length      = 264.0 cm
  ICS Outer Length      = 288.0 cm
  ICS Inner EndCap Surf = 5.391e+04 cm**2
  ICS Outer EndCap Surf = 6.424e+04 cm**2
  ICS Volume            = 4.269e+06 cm**3
  ICS Mass              = 3.825e+04 kg

* VESSEL
  VESSEL Thickness    = 2.0 cm
  VESSEL Inner Rad    = 143.0 cm
  VESSEL Outer Rad    = 145.0 cm
  VES

In [11]:
mat_act = get_activities(BKGND_LEVEL)

In [12]:
if VERBOSE:
    print_activities(BKGND_LEVEL)

*** 'pessimistic' MATERIAL ACTIVITIES ***

* Copper:
    Tl208: 0.00147 mBq/kg
    Bi214: 0.012 mBq/kg

* DiceBoard:
    Tl208: 1.04 mBq/m**2
    Bi214: 7.0 mBq/m**2

* HDPE:
    Tl208: 0.00755 mBq/kg
    Bi214: 0.062 mBq/kg

* SSteel316Ti:
    Tl208: 0.0431 mBq/kg
    Bi214: 0.46 mBq/kg


# COMPUTING THE DETECTOR BACKGROUND LEVEL

#### Radiogenic contributions

In principle. only the contributions from the 'READOUT_PLANES', 'FIELD_CAGE' and 'INNER_SHIELDING' are considered.

Backgrounds expressed in Becquerel

In [13]:
radiogenic_bkgnd = get_background_level(DETECTOR, BKGND_LEVEL)
radiogenic_bkgnd['Total'] = radiogenic_bkgnd['Bi214'] + radiogenic_bkgnd['Tl208']
radiogenic_bkgnd

,Bi214,Tl208,Total
source,,,
READOUT_PLANE,0.074330,0.011043,0.085373
CATHODE,0.004878,0.000457,0.005336
FIELD_CAGE,0.012821,0.001561,0.014382
INNER_SHIELDING,0.458984,0.056225,0.515209


In [14]:
if VERBOSE:
    tot_bkgnd = radiogenic_bkgnd['Total'].sum()
    print(f"Radiogenic '{DETECTOR}' background: {tot_bkgnd:.4} Bq.")
    
    tot_Bi = radiogenic_bkgnd['Bi214'].sum()
    tot_Tl = radiogenic_bkgnd['Tl208'].sum()
    print(f"Total Bi214 background: {tot_Bi:8.4} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
    print(f"\nTotal Tl208 background: {tot_Tl:8.4} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")
    
    tot_READOUT_PLANE = radiogenic_bkgnd.loc['READOUT_PLANE'].Total
    tot_CATHODE       = radiogenic_bkgnd.loc['CATHODE'].Total
    tot_FIELD_CAGE    = radiogenic_bkgnd.loc['FIELD_CAGE'].Total
    tot_ICS           = radiogenic_bkgnd.loc['INNER_SHIELDING', 'Total']
    print(f"\nTotal READOUT_PLANE background: {tot_READOUT_PLANE:8.4} Bq.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:.4} %")
    print(f"Total CATHODE background:       {tot_CATHODE:8.4} Bq.  ->  {tot_CATHODE/tot_bkgnd*100:.4} %")
    print(f"Total FIELD_CAGE background:    {tot_FIELD_CAGE:8.4} Bq.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:.4} %")
    print(f"Total ICS background:           {tot_ICS:8.4} Bq.  ->  {tot_ICS/tot_bkgnd*100:.4} %")


Radiogenic 'next_hd' background: 0.6203 Bq.
Total Bi214 background:    0.551 Bq.  ->  88.83 %

Total Tl208 background:  0.06929 Bq.  ->  11.17 %

Total READOUT_PLANE background:  0.08537 Bq.  ->  13.76 %
Total CATHODE background:       0.005336 Bq.  ->  0.8602 %
Total FIELD_CAGE background:     0.01438 Bq.  ->  2.319 %
Total ICS background:             0.5152 Bq.  ->  83.06 %


#### Radon contribution

Its final contribution comes from an extra Bi214 concentration in the central CATHODE

In [ ]:
# TO BE IMPLEMENTED

#### Muons contribution

Xe137 decay is the only muon contribution considered.

In [ ]:
# TO BE IMPLEMENTED

# LOADING & SELECTING REJECTION FACTORS

In [15]:
rej_factors = get_rejection_factors(DETECTOR)

In [16]:
if VERBOSE:
    print(rej_factors)

                                         bb0nu     Xe137         Tl208  \
source          energyRes spatialDef                                     
ACTIVE          0.7       10x10x10    0.289450  0.000149           NaN   
                0.5       3x3x3       0.245723  0.000052           NaN   
READOUT_PLANE   0.7       10x10x10         NaN       NaN  3.117144e-07   
                0.5       3x3x3            NaN       NaN  1.006543e-07   
CATHODE         0.7       10x10x10         NaN       NaN           NaN   
                0.5       3x3x3            NaN       NaN           NaN   
FIELD_CAGE      0.7       10x10x10         NaN       NaN  2.006018e-07   
                0.5       3x3x3            NaN       NaN  4.087890e-08   
INNER_SHIELDING 0.7       10x10x10         NaN       NaN  1.333333e-07   
                0.5       3x3x3            NaN       NaN  0.000000e+00   

                                             Bi214  
source          energyRes spatialDef                
ACTIV

### OPTIONS TO STUDY

In [17]:
energyRes  = 0.7
spatialDef = '10x10x10'

#### Signal efficiency

In [18]:
sig_eff = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'bb0nu']
sig_eff

0.28945

#### Radiogenic rejection_factors

In [19]:
radiogenic_rejections = rej_factors.loc[pd.IndexSlice[:, energyRes, spatialDef], ['Bi214', 'Tl208']]
radiogenic_rejections.drop('ACTIVE', level=0, axis=0, inplace=True)
radiogenic_rejections

,,,Bi214,Tl208
source,energyRes,spatialDef,,
READOUT_PLANE,0.7,10x10x10,3.900000e-07,3.117144e-07
CATHODE,0.7,10x10x10,3.882476e-07,NaN
FIELD_CAGE,0.7,10x10x10,5.400000e-07,2.006018e-07
INNER_SHIELDING,0.7,10x10x10,6.206206e-08,1.333333e-07


#### Radon rejection_factor

As in previos NEXT detectors we assume that the whole amount of Radon contamination is translated to Bi214 contamination from the CATHODE.

In [36]:
radon_rejection  = rej_factors.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef], 'Bi214']

In [37]:
if VERBOSE:
    print(f"Radon rejection factor:       {radon_rejection:.4}")

Radon rejection factor:       3.882e-07


#### Muons rejection_factors

The muons rejection factor = Xe137_activation * Xe137_rejection

In [31]:
#Xe137_activation = XXXXX
Xe137_rejection = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'Xe137']
#muon_rejection = Xe137_activation * Xe137_rejection

In [35]:
if VERBOSE:
    print(f"Xe137 activation probability: XXXXX")
    print(f"Xe137 rejection factor:       {Xe137_rejection:.4}")
    print(f"\nMuons rejection factor:       XXXXX")

Xe137 activation probability: XXXXX
Xe137 rejection factor:       0.0001486

Muons rejection factor:       XXXXX


# COMPUTING THE DETECTOR BACKGROUND INDEX

#### Estimating the Bq -> ckky conversion factor 

In [23]:
secs_in_year    = 60 * 60 * 24 * 365
Xe136_abundance = 0.9026
Xe136_mass_kg   = det_dim['ACTIVE_mass'] * Xe136_abundance / units.kg
ROI_settings    = get_roi_settings(energyRes)

In [24]:
toCKKY = secs_in_year / Xe136_mass_kg / ((ROI_settings['Emax'] - ROI_settings['Emin']) / units.keV)

In [25]:
if VERBOSE:
    print(f"Xe136 mass: {Xe136_mass_kg:.4} kg.")
    print(f"ROI: [{ROI_settings['Emin']/units.keV:.4}, {ROI_settings['Emax']/units.keV:.4}] keV.")
    print(f"toCKKY factor: {toCKKY:.4}")

Xe136 mass: 1.109e+03 kg.
ROI: [2.453e+03, 2.475e+03] keV.
toCKKY factor: 1.293e+03


#### Radiogenic background index

In [26]:
radiogenic_bkgnd_index_Bq = radiogenic_bkgnd[['Bi214', 'Tl208']] * radiogenic_rejections[['Bi214', 'Tl208']]
radiogenic_bkgnd_index_Bq['Total'] = radiogenic_bkgnd_index_Bq['Bi214'].add(radiogenic_bkgnd_index_Bq['Tl208'],
                                                                            fill_value = 0.)
radiogenic_bkgnd_index = radiogenic_bkgnd_index_Bq * toCKKY
radiogenic_bkgnd_index

,,,Bi214,Tl208,Total
source,energyRes,spatialDef,,,
READOUT_PLANE,0.7,10x10x10,0.000037,4.449851e-06,0.000042
CATHODE,0.7,10x10x10,0.000002,NaN,0.000002
FIELD_CAGE,0.7,10x10x10,0.000009,4.048610e-07,0.000009
INNER_SHIELDING,0.7,10x10x10,0.000037,9.690824e-06,0.000047


In [27]:
if VERBOSE:
    tot_bkgnd = radiogenic_bkgnd_index['Total'].sum()
    print(f"Radiogenic '{DETECTOR}' background index: {tot_bkgnd:.4} ckky")
    
    tot_Bi = radiogenic_bkgnd_index['Bi214'].sum()
    tot_Tl = radiogenic_bkgnd_index['Tl208'].sum()
    print(f"\nTotal Bi214 background index: {tot_Bi:8.4} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
    print(f"Total Tl208 background index: {tot_Tl:8.4} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")
    
    tot_READOUT_PLANE = radiogenic_bkgnd_index.loc[pd.IndexSlice['READOUT_PLANE', energyRes, spatialDef]].Total
    tot_CATHODE       = radiogenic_bkgnd_index.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef]].Total
    tot_FIELD_CAGE    = radiogenic_bkgnd_index.loc[pd.IndexSlice['FIELD_CAGE', energyRes, spatialDef]].Total
    tot_ICS           = radiogenic_bkgnd_index.loc[pd.IndexSlice['INNER_SHIELDING', energyRes, spatialDef]].Total
    print(f"\nTotal READOUT_PLANE background index: {tot_READOUT_PLANE:8.4} ckky.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:.4} %")
    print(f"Total CATHODE       background index: {tot_CATHODE:8.4} ckky.  ->  {tot_CATHODE/tot_bkgnd*100:.4} %")
    print(f"Total FIELD_CAGE    background index: {tot_FIELD_CAGE:8.4} ckky.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:.4} %")
    print(f"Total ICS           background index: {tot_ICS:8.4} ckky.  ->  {tot_ICS/tot_bkgnd*100:.4} %")


Radiogenic 'next_hd' background index: 0.0001002 ckky

Total Bi214 background index: 8.569e-05 Bq.  ->  85.49 %
Total Tl208 background index: 1.455e-05 Bq.  ->  14.51 %

Total READOUT_PLANE background index: 4.192e-05 ckky.  ->  41.82 %
Total CATHODE       background index: 2.448e-06 ckky.  ->  2.443 %
Total FIELD_CAGE    background index: 9.355e-06 ckky.  ->  9.332 %
Total ICS           background index: 4.651e-05 ckky.  ->  46.4 %


#### Radon background index

In [ ]:
# TO BE IMPLEMENTED

#### Muons background index

In [ ]:
# TO BE IMPLEMENTED

# SENSITIVITY

In [ ]:
#from pybbsens import isotope